<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [3]:
df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 9.15000000e+02
  3.40000000e+01 5.10699074e+00 1.00000000e+00 4.03600000e+03
  8.70081028e+01 2.43400000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 9.15000000e+02 3.40000000e+01 5.10699074e+00
  1.00000000e+00 4.03600000e+03 8.70081028e+01 2.43400000e+03
  0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.10000000e+01 7.00000000e+00 4.51000000e+02
  6.00000000e+00 4.34344907e+00 1.00000000e+00 6.40500000e+03
  1.53869726e+02 5.22200000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.10000000e+01
  7.00000000e+00 4.51000000e+02 6.00000000e+00 4.34344907e+00
  1.00000000e+00 6.40500000e+03 1.53869726e+02 5.22200000e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [6]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 9.15000000e+02
  3.40000000e+01 5.10699074e+00 1.00000000e+00 4.03600000e+03
  8.70081028e+01 2.43400000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 9.15000000e+02 3.40000000e+01 5.10699074e+00
  1.00000000e+00 4.03600000e+03 8.70081028e+01 2.43400000e+03
  0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.10000000e+01 7.00000000e+00 4.51000000e+02
  6.00000000e+00 4.34344907e+00 1.00000000e+00 6.40500000e+03
  1.53869726e+02 5.22200000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.10000000e+01
  7.00000000e+00 4.51000000e+02 6.00000000e+00 4.34344907e+00
  1.00000000e+00 6.40500000e+03 1.53869726e+02 5.22200000e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [8]:
print(mean_train)
print(std_train)

[2.77476080e-01 1.60739591e+00 2.51771399e+00 3.91052495e+00
 7.36091560e-01 5.29580553e+01 2.43177657e+01 7.08759211e+02
 1.90589604e+01 8.25230425e+01 1.90918024e+00 1.74726636e+03
 8.53276442e+01 9.02828547e+02 2.77476080e-01 1.60739591e+00
 2.51771399e+00 3.91052495e+00 7.36091560e-01 5.29580553e+01
 2.43177657e+01 7.08759211e+02 1.90589604e+01 8.25230425e+01
 1.90918024e+00 1.74726636e+03 8.53276442e+01 9.02828547e+02
 2.77476080e-01 1.58210499e-01 3.50142229e-02 4.79958624e-01
 1.54641841e-02 3.38763900e-02]
[4.47753397e-01 1.64389526e+00 6.21038647e+00 2.39289237e+01
 1.11867691e+00 7.30512812e+02 5.16431750e+02 1.06995963e+03
 2.68694733e+01 2.14019615e+02 7.43961996e+00 1.98299249e+03
 8.19211779e+01 1.31232638e+03 4.47753397e-01 1.64389526e+00
 6.21038647e+00 2.39289237e+01 1.11867691e+00 7.30512812e+02
 5.16431750e+02 1.06995963e+03 2.68694733e+01 2.14019615e+02
 7.43961996e+00 1.98299249e+03 8.19211779e+01 1.31232638e+03
 4.47753397e-01 3.64938265e-01 1.83815742e-01 4.99598

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [10]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-0.61970737 -0.36948577 -0.24438318 -0.12163209 -0.65800193 -0.07112545
  -0.04515169  0.19275567  0.55606001 -0.3617241  -0.12220789  1.1541817
   0.02051312  1.16676116 -0.61970737 -0.36948577 -0.24438318 -0.12163209
  -0.65800193 -0.07112545 -0.04515169  0.19275567  0.55606001 -0.3617241
  -0.12220789  1.1541817   0.02051312  1.16676116 -0.61970737  2.30666275
  -0.19048544 -0.96068929 -0.1253279  -0.18725448]
 [-0.61970737 -0.36948577 -0.24438318 -0.12163209 -0.65800193 -0.05743644
  -0.0335335  -0.24090555 -0.48601475 -0.36529172 -0.12220789  2.34884079
   0.83668331  3.29123266 -0.61970737 -0.36948577 -0.24438318 -0.12163209
  -0.65800193 -0.05743644 -0.0335335  -0.24090555 -0.48601475 -0.36529172
  -0.12220789  2.34884079  0.83668331  3.29123266 -0.61970737 -0.43352675
  -0.19048544  1.04091927 -0.1253279  -0.18725448]
 [ 1.61366485  0.23882549 -0.0833626  -0.07984166  0.20994358 -0.06701875
  -0.04515169  1.63206232  2.04473824  0.02698049  0.01220758 -0.88112606
  -1.0415822

In [0]:
def create_sampler(y):
  _, counts = np.unique(y, return_counts=True)
  n_samples = len(y)
  class_weights = n_samples / counts
  weights = class_weights[y]
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [15]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[ 1.6137, -0.3695, -0.2444, -0.1216, -0.6580, -0.0711, -0.0452,  0.4984,
          0.1095, -0.3819, -0.1222,  0.0281,  0.5759,  0.1236,  1.6137, -0.3695,
         -0.2444, -0.1216, -0.6580, -0.0711, -0.0452,  0.4984,  0.1095, -0.3819,
         -0.1222,  0.0281,  0.5759,  0.1236,  1.6137, -0.4335, -0.1905, -0.9607,
         -0.1253, -0.1873],
        [ 1.6137, -0.3695, -0.2444, -0.1216, -0.6580, -0.0670, -0.0432,  1.3283,
         -0.5232, -0.3828, -0.1222, -0.5957, -0.1207, -0.5881,  1.6137, -0.3695,
         -0.2444, -0.1216, -0.6580, -0.0670, -0.0432,  1.3283, -0.5232, -0.3828,
         -0.1222, -0.5957, -0.1207, -0.5881,  1.6137, -0.4335, -0.1905, -0.9607,
         -0.1253, -0.1873],
        [-0.6197, -0.3695, -0.2444, -0.0798, -0.3421, -0.0383, -0.0374,  0.0072,
         -0.5232, -0.3556,  0.0122,  1.2071,  2.0447,  2.0187, -0.6197, -0.3695,
         -0.2444, -0.0798, -0.3421, -0.0383, -0.0374,  0.0072, -0.5232, -0.3556,
          0.0122,  1.2071,  2.0447,  2.0187, -0.6197,

In [0]:
class Classifier(nn.Module):
  def __init__(self, epochs, input_size, hidden_size, drop_prob, val_loss=None):
    super(Classifier, self).__init__()
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.relu(self.fc2(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

In [17]:
classifier = Classifier(epochs=1000, input_size=x.shape[1], hidden_size=256, drop_prob=0.5)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fcout): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'epochs': classifier.epochs,
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'drop_prob': classifier.drop_prob,
      'val_loss': classifier.val_loss,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['epochs'], checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['drop_prob'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [20]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.4988],
        [0.4987],
        [0.5096]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [22]:
train_loss = 0
epochs = classifier.epochs
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.val_loss = val_loss
      classifier.epochs = epoch
      save_classifier(classifier)

Epoch: 0, Train loss: 0.6089057138091639, Val loss: 0.6004581753360597
Epoch: 1, Train loss: 0.5770850898156206, Val loss: 0.5182919823809674
Epoch: 2, Train loss: 0.5619035633021531, Val loss: 0.533738300204277
Epoch: 3, Train loss: 0.5496999628491933, Val loss: 0.514270973048712
Epoch: 4, Train loss: 0.535005087149634, Val loss: 0.5310197033380207
Epoch: 5, Train loss: 0.5339375379967192, Val loss: 0.5372801466207755
Epoch: 6, Train loss: 0.5296086981194258, Val loss: 0.5193872596872481
Epoch: 7, Train loss: 0.524873953938215, Val loss: 0.5392486723630052
Epoch: 8, Train loss: 0.5166220785724418, Val loss: 0.528553941924321
Epoch: 9, Train loss: 0.5241742819588103, Val loss: 0.5215287200714412
Epoch: 10, Train loss: 0.5141183140247556, Val loss: 0.5139651839670382
Epoch: 11, Train loss: 0.5132377124208356, Val loss: 0.521879185579325
Epoch: 12, Train loss: 0.5092885509300079, Val loss: 0.5290270306562123
Epoch: 13, Train loss: 0.5086532596307048, Val loss: 0.512881329969356
Epoch: 14

In [0]:
def calculate_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  return np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1))

def gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calculate_recalls(targets, predictions)

    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [24]:
classifier = load_classifier()
if torch.cuda.is_available():
  classifier = classifier.cuda()
train_accuracy = gmean(classifier, train_dataloader)
test_accuracy = gmean(classifier, test_dataloader)
print('Epochs: {}'.format(classifier.epochs))
print('Train g-mean: {}'.format(train_accuracy.item()))
print('Test g-mean: {}'.format(test_accuracy.item()))

Epochs: 480
Train g-mean: 0.8387829745989959
Test g-mean: nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
